In [61]:
def get_weather_data():
    import requests
    from datetime import datetime, timedelta
    import pandas as pd
    from xml.etree import ElementTree as ET

    # 기상청 API 요청 URL
    url = "http://apis.data.go.kr/1360000/VilageFcstInfoService_2.0/getVilageFcst"
    service_key = "pRdsQgsK15J0SEpvM%2Fiu9qO2Hxpw3OXpj8sCHKKP0G3YyK2BwM60RmheVzPTyeyE2bb0htzlZOzIouMwJst4Tw%3D%3D"  # 발급받은 인증키 입력
    base_date = datetime.now().strftime("%Y%m%d")  # 현재 날짜
    current_time = datetime.now().strftime("%H%M")
    hour_list = ["0200", "0500", "0800", "1100", "1400", "1700", "2000", "2300"]
    base_time = max(filter(lambda x: x < current_time, hour_list))
    nx = "66"  # 충남대학교 위도
    ny = "101"  # 충남대학교 경도
    num_of_rows = "2000"  # 한 페이지 결과 수
    page_no = "1"  # 페이지 번호
    output_type = "xml"  # 응답 데이터 형식

    # 데이터를 담을 리스트 초기화
    data_list = []

    req_url = f"{url}?serviceKey={service_key}&" \
            f"dataType={output_type}&" \
            f"base_date={base_date}&" \
            f"base_time={base_time}&" \
            f"nx={nx}&ny={ny}&" \
            f"numOfRows={num_of_rows}&pageNo={page_no}"

    # API 호출
    res = requests.get(req_url)
    # XML 데이터 파싱
    root = ET.fromstring(res.text)
    # 필요한 데이터 추출
    items = root.find("body").find("items").findall("item")
    # 필요한 데이터를 추출하여 리스트에 추가
    for item in items:
        data_dict = {}
        data_dict["기준일자"] = item.findtext("baseDate")
        data_dict["기준시간"] = item.findtext("baseTime")[:2] + ":00"
        data_dict["구분"] = item.findtext("category")
        data_dict["값"] = item.findtext("fcstValue")
        data_dict["예측시간"] = item.findtext("fcstTime")[:2] + ":00"
        data_dict["예측일자"] = item.findtext("fcstDate")
        data_list.append(data_dict)
                
    # 데이터프레임 생성
    df = pd.DataFrame(data_list)

    # 날짜 형식 변환
    df["기준일자"] = pd.to_datetime(df["기준일자"], format="%Y%m%d")
    df["예측일자"] = pd.to_datetime(df["예측일자"], format="%Y%m%d")

    # 피벗 테이블 생성
    df_pivot = pd.pivot_table(df, index=["기준일자", "기준시간","예측일자", "예측시간"], columns=["구분"], values=["값"], aggfunc=min)

    # 칼럼 이름 정리
    df_pivot.columns = df_pivot.columns.get_level_values(1)
    
    # 인덱스 리셋
    df_pivot.reset_index(inplace=True)
    
    # 칼럼 이름 정리
    df_pivot=df_pivot.rename(columns={"PCP": "강수량(PCP)", "POP": "강수확률(POP)", "PTY": "강수형태(PTY)", "SNO": "눈량(SNO)", "REH": "습도(REH)", "TMP": "기온(TMP)", "SKY": "하늘상태(SKY)", "TMN": "아침 최저기온(TMN)", "TMX": "낮 최고기온(TMX)", "UUU": "동서바람성분(UUU)", "VEC": "풍향(VEC)", "VVV": "남북바람성분(VVV)", "WAV": "파고(WAV)", "WSD": "풍속(WSD)"})

    return df_pivot


In [62]:
df_weather = get_weather_data()
df_weather.head(79)

구분,기준일자,기준시간,예측일자,예측시간,강수량(PCP),강수확률(POP),강수형태(PTY),습도(REH),하늘상태(SKY),눈량(SNO),아침 최저기온(TMN),기온(TMP),낮 최고기온(TMX),동서바람성분(UUU),풍향(VEC),남북바람성분(VVV),파고(WAV),풍속(WSD)
0,2023-02-20,17:00,2023-02-20,18:00,강수없음,0,0,40,1,적설없음,NaN,-1,NaN,2.1,309,-1.7,0,2.7
1,2023-02-20,17:00,2023-02-20,19:00,강수없음,0,0,45,1,적설없음,NaN,-2,NaN,2.1,302,-1.3,0,2.5
2,2023-02-20,17:00,2023-02-20,20:00,강수없음,0,0,50,1,적설없음,NaN,-3,NaN,2,309,-1.6,0,2.6
3,2023-02-20,17:00,2023-02-20,21:00,강수없음,0,0,55,1,적설없음,NaN,-4,NaN,1.5,324,-2.1,0,2.6
4,2023-02-20,17:00,2023-02-20,22:00,강수없음,0,0,55,1,적설없음,NaN,-4,NaN,1.2,334,-2.5,0,2.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,2023-02-20,17:00,2023-02-23,20:00,강수없음,30,0,60,4,적설없음,NaN,3,NaN,0.6,333,-1.2,0,1.3
75,2023-02-20,17:00,2023-02-23,21:00,강수없음,30,0,65,4,적설없음,NaN,2,NaN,0.4,340,-1.1,0,1.2
76,2023-02-20,17:00,2023-02-23,22:00,강수없음,30,0,70,4,적설없음,NaN,1,NaN,0.5,331,-0.9,0,1
77,2023-02-20,17:00,2023-02-23,23:00,강수없음,30,0,75,4,적설없음,NaN,0,NaN,0.5,328,-0.8,0,0.9


In [63]:
df_weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79 entries, 0 to 78
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   기준일자          79 non-null     datetime64[ns]
 1   기준시간          79 non-null     object        
 2   예측일자          79 non-null     datetime64[ns]
 3   예측시간          79 non-null     object        
 4   강수량(PCP)      79 non-null     object        
 5   강수확률(POP)     79 non-null     object        
 6   강수형태(PTY)     79 non-null     object        
 7   습도(REH)       79 non-null     object        
 8   하늘상태(SKY)     79 non-null     object        
 9   눈량(SNO)       79 non-null     object        
 10  아침 최저기온(TMN)  3 non-null      object        
 11  기온(TMP)       79 non-null     object        
 12  낮 최고기온(TMX)   3 non-null      object        
 13  동서바람성분(UUU)   79 non-null     object        
 14  풍향(VEC)       79 non-null     object        
 15  남북바람성분(VVV)   79 non-null     object      